<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C3/W4/ungraded_labs/C3_W4_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Next Word Prediction**

## Install tensorflowjs

In [ ]:
!pip install tensorflowjs

## Import Libraries

In this section, we need to import all the required libraries. Here all those libraries.

In [ ]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow import keras
from keras.utils.vis_utils import plot_model

## Building the Word Vocabulary
 To get all the word vocabulary from destination list, here we use the data that we've gathered and saved on Google Sheet and simply inserted it in destination variable. 

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Fadlanbima/ML_repo/master/nwp/Destination.csv')

In [ ]:
df.head(10)

In [ ]:
df = df['Destinasi']

Generate each word index dictionary and compute the total words

In [ ]:
# Initialize Tokenizer class
tokenizer = Tokenizer()

# Generate the word index dictionary
tokenizer.fit_on_texts(df)

# Define the total words. Add 1 for the index `0` as the padding token.
total_words = len(tokenizer.word_index) + 1

print(f'word index dictionary: {tokenizer.word_index}')
print(f'total words: {total_words}')

## Preprocessing the Dataset

Here we will generate the training sequences which the result would be inputs as padded sequences and the labels will be labelled as one-hot encoded arrays.

In [ ]:
# Initialize the sequences list
input_sequences = []

# Loop over every line
for line in df:

    # Tokenize the current line
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Loop over the line several times to generate the subphrases
    for i in range(1, len(token_list)):
        
        # Generate the subphrase
        n_gram_sequence = token_list[:i+1]

        # Append the subphrase to the sequences list
        input_sequences.append(n_gram_sequence)

# Get the length of the longest line
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding='pre'))

# Create inputs and label by splitting the last token in the subphrases
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

# Convert the label into one-hot arrays
ys = tf.keras.utils.to_categorical(labels, num_classes = total_words)

## Build the Model

In [ ]:
model = Sequential([
          Embedding(total_words, 64, input_length = max_sequence_len-1),
          LSTM(100, return_sequences=True),
          LSTM(100),
          Dense(50, activation='relu'),
          Dense(total_words, activation = 'softmax')])

model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.002),
              metrics = ['accuracy'])

## Show the Model Summary

In [ ]:
model.summary()

## Plot the Model

In [ ]:
keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

## Train the Model

In [ ]:
history = model.fit(xs, ys, epochs = 200)

## Visual the Result

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

# Visualize the accuracy
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

## Make Prediction

In [ ]:
vocab_array = np.array(list(tokenizer.word_index.keys()))

def make_prediction(text, n_words):
    for i in range(n_words):
        text_tokenize = tokenizer.texts_to_sequences([text])
        text_padded = tf.keras.preprocessing.sequence.pad_sequences(text_tokenize, maxlen = max_sequence_len-1)
        prediction = np.squeeze(np.argmax(model.predict(text_padded), axis = -1))
        prediction = str(vocab_array[prediction - 1])
        text += " " + prediction.capitalize()
    return text

In [ ]:
#text_new = input("Enter your destination: ")

In [ ]:
make_prediction("Taman Mini", 2)

## Save the Model

In [ ]:
saved_model_path = "nwp_model.h5"

model.save(saved_model_path)

To JSON

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

To pickle

In [ ]:
import pickle

In [ ]:
file_name='nwp_model.pkl'
pickle.dump(model, open(file_name, 'wb')) 

If wanna load the model

In [ ]:
loaded_model = pickle.load(open(file_name, 'rb'))
result = loaded_model(xs, ys)
print(result)

To Pb

In [ ]:
import os
import tempfile

tmpdir = tempfile.mkdtemp()

In [ ]:
mobilenet_save_path = os.path.join(tmpdir, "SavedModel")
tf.saved_model.save(model, mobilenet_save_path)

In [ ]:
ls {mobilenet_save_path}

To TF Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='/tmp/tmpalaqkqms/SavedModel', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

In [ ]:
# Show model size in KBs.
quantized_model_size = len(tflite_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

In [ ]:
# Save the quantized model to file to the Downloads directory
f = open('nwp_model.tflite', "wb")
f.write(tflite_model)
f.close()

# Download the digit classification model
from google.colab import files
files.download('nwp_model.tflite')

print('`nwp_model.tflite` has been downloaded')